### // **Imports**

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from sklearn.model_selection import train_test_split

### // **Load Device**

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


### // **Class Definition**

In [ ]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.all_layers = nn.Sequential(
        nn.Linear(102, 102),
        nn.ReLU,

        nn.Linear(102, 34),
        nn.ReLU,

        nn.Linear(34, 2),
    )

  def forward(self, x):
    logits = self.all_layers(x)
    return logits
